In [1]:
import json

import numpy as np
import tensorflow as tf
import os
import pandas as pd
from DataGenerator import DataGenerator
from metrics import *
import random

In [2]:
OCTAVES = 8
TEST_INDEX = 2

In [3]:
model_folder = f"saved/2024-03-15/2_1_{OCTAVES}_octaves"
model_path = f"{model_folder}/model"
checkpoint_path = f"{model_folder}/checkpoints"
model = tf.keras.models.load_model(model_path)

In [4]:
ids_path = "../../data/archived/id_22050.csv"
list_IDs = list(pd.read_csv(ids_path, header=None)[0])
testing_ids = [id for id in list_IDs if int(id.split("_")[0]) == TEST_INDEX]

In [5]:
test_generator = DataGenerator(testing_ids,
                               batch_size=400,
                               data_path=f"../../data/archived/GuitarSet/{OCTAVES}_octaves/",
                               shuffle=False,
                               con_win_size=9)

In [6]:
print(testing_ids)

['02_Rock3-148-C_solo_0', '02_Rock3-148-C_solo_1', '02_Rock3-148-C_solo_2', '02_Rock3-148-C_solo_3', '02_Rock3-148-C_solo_4', '02_Rock3-148-C_solo_5', '02_Rock3-148-C_solo_6', '02_Rock3-148-C_solo_7', '02_Rock3-148-C_solo_8', '02_Rock3-148-C_solo_9', '02_Rock3-148-C_solo_10', '02_Rock3-148-C_solo_11', '02_Rock3-148-C_solo_12', '02_Rock3-148-C_solo_13', '02_Rock3-148-C_solo_14', '02_Rock3-148-C_solo_15', '02_Rock3-148-C_solo_16', '02_Rock3-148-C_solo_17', '02_Rock3-148-C_solo_18', '02_Rock3-148-C_solo_19', '02_Rock3-148-C_solo_20', '02_Rock3-148-C_solo_21', '02_Rock3-148-C_solo_22', '02_Rock3-148-C_solo_23', '02_Rock3-148-C_solo_24', '02_Rock3-148-C_solo_25', '02_Rock3-148-C_solo_26', '02_Rock3-148-C_solo_27', '02_Rock3-148-C_solo_28', '02_Rock3-148-C_solo_29', '02_Rock3-148-C_solo_30', '02_Rock3-148-C_solo_31', '02_Rock3-148-C_solo_32', '02_Rock3-148-C_solo_33', '02_Rock3-148-C_solo_34', '02_Rock3-148-C_solo_35', '02_Rock3-148-C_solo_36', '02_Rock3-148-C_solo_37', '02_Rock3-148-C_solo_

In [7]:
def test(model, test_ids):
    y_gt = np.empty((len(test_ids), 6, 21))
    y_pred = np.empty((len(test_ids), 6, 21))
    index = 0
    for i in range(len(test_generator)):
        X_test, gt = test_generator[i]
        pred = model.predict(X_test, verbose=0)
        size = len(pred[0])
        for sample_index in range(size):
            EString_pred = pred[0][sample_index]
            AString_pred = pred[1][sample_index]
            DString_pred = pred[2][sample_index]
            GString_pred = pred[3][sample_index]
            BString_pred = pred[4][sample_index]
            eString_pred = pred[5][sample_index]
            sample_tab_pred = np.array([EString_pred, AString_pred, DString_pred, GString_pred, BString_pred, eString_pred])
            y_pred[index,] = sample_tab_pred

            EString_gt = gt["EString"][sample_index]
            AString_gt = gt["AString"][sample_index]
            DString_gt = gt["DString"][sample_index]
            GString_gt = gt["GString"][sample_index]
            BString_gt = gt["BString"][sample_index]
            eString_gt = gt["eString"][sample_index]
            sample_tab_gt = np.array([EString_gt, AString_gt, DString_gt, GString_gt, BString_gt, eString_gt])
            y_gt[index,] = sample_tab_gt
            index += 1
    return y_gt, y_pred


In [8]:
def save_predictions(predictions, ground_truths, save_path, filename):
    np.savez(f"{save_path}/{filename}.npz", y_pred = predictions, y_gt = ground_truths)

In [9]:
def evaluate(predictions, ground_truths):
    metrics = {
        "pitch_precision" : [],
        "pitch_recall" : [],
        "pitch_f_score" : [],
        "tab_precision" : [],
        "tab_recall" : [],
        "tab_f_score" : []
    }
    metrics["pitch_precision"].append(pitch_precision(predictions, ground_truths))
    metrics["pitch_recall"].append(pitch_recall(predictions, ground_truths))
    metrics["pitch_f_score"].append(pitch_f_score(predictions, ground_truths))
    metrics["tab_precision"].append(tab_precision(predictions, ground_truths))
    metrics["tab_recall"].append(tab_recall(predictions, ground_truths))
    metrics["tab_f_score"].append(tab_f_measure(predictions, ground_truths))
    return metrics

In [10]:
def test_epochs(model, checkpoints_path):
    result = {}
    for i, epoch_file in enumerate(os.listdir(checkpoints_path)):
        npz_already_created = f"predictions_{i+1}_epoch.npz" in os.listdir(model_folder)
        model.load_weights(f"{checkpoints_path}/{epoch_file}")
        #check if npz file already exists
        if npz_already_created:
            print(f"Loading epoch {i+1}...")
            archive = np.load(f"{model_folder}/predictions_{i+1}_epoch.npz")
            y_gt = archive["y_gt"]
            y_pred = archive["y_pred"]
        else:
            print(f"Testing epoch {i+1}...")
            y_gt, y_pred = test(model, testing_ids)
        print(f"Loaded predictions...")
        if not npz_already_created:
            print("Saving predictions...")
            save_predictions(y_pred, y_gt, model_folder, f"predictions_{i+1}_epoch")
        print(f"Evaluating...")
        metrics = evaluate(y_pred, y_gt)
        print(metrics)
        result[str(i+1)] = metrics
    print("Dumping in json file...")
    with open(f"{model_folder}/pred_by_epoch.json", "w") as f:
        json.dump(result, f, indent=4)

In [11]:
test_epochs(model, checkpoint_path)

Testing epoch 1...
Loaded predictions...
Saving predictions...
Evaluating...
{'pitch_precision': [0.8920332064394808], 'pitch_recall': [0.7339577641251679], 'pitch_f_score': [0.8053116032942872], 'tab_precision': [0.7567710776393739], 'tab_recall': [0.6568279314530417], 'tab_f_score': [0.7032664543652956]}
Testing epoch 2...
Loaded predictions...
Saving predictions...
Evaluating...
{'pitch_precision': [0.8940314591275842], 'pitch_recall': [0.7451431762947757], 'pitch_f_score': [0.8128254632126231], 'tab_precision': [0.7806304814046344], 'tab_recall': [0.6750880208686736], 'tab_f_score': [0.7240332329337564]}
Testing epoch 3...
Loaded predictions...
Saving predictions...
Evaluating...
{'pitch_precision': [0.9021651524032825], 'pitch_recall': [0.7434449861880538], 'pitch_f_score': [0.8151507377585052], 'tab_precision': [0.7950415563954363], 'tab_recall': [0.6771990138647004], 'tab_f_score': [0.7314040501917611]}
Testing epoch 4...
Loaded predictions...
Saving predictions...
Evaluating...